In [22]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import pickle
from scipy import stats
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split

fileweights = 'drive/MyDrive/Colab Notebooks/weights/w.hdf5'
RANDOM_SEED = 42

# load the dataset, returns train and test X and y elements
def load_WISDM_dataset(prefix=''):
  columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
  df = pd.read_csv(prefix + 'WISDM/WISDM_ar_v1.1_raw.txt', header = None, names = columns)
  df = df.dropna()

  N_TIME_STEPS = 200
  N_FEATURES = 3
  step = 20
  segments = []
  labels = []
  
  for i in range(0, len(df) - N_TIME_STEPS, step):
    xs = df['x-axis'].values[i: i + N_TIME_STEPS]
    ys = df['y-axis'].values[i: i + N_TIME_STEPS]
    zs = df['z-axis'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
    segments.append([xs, ys, zs])
    labels.append(label)
  
  print(np.array(segments).shape)

  reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
  labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)

  X_train, X_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size=0.2, random_state=RANDOM_SEED)
  return X_train, y_train, X_test, y_test

def method_to_create_the_model(n_length, n_features, n_outputs):
	# define model
  model = Sequential()
  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
  model.add(TimeDistributed(Dropout(0.5)))
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(100))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  return model

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 50, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 20, 200

  print(trainX.shape)
  #questo 1 da dove viene?
  trainX = trainX.reshape(trainX.shape[0], 1, n_length, n_features)
  testX = testX.reshape(testX.shape[0], 1, n_length, n_features)

  model = method_to_create_the_model(n_length, n_features, n_outputs)
    
  save_weights = keras.callbacks.ModelCheckpoint(fileweights, save_weights_only=True) #, period=5)
  
  earlyStopping = keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min')

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[save_weights, earlyStopping])
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy
	
def evaluate_network(testX):
  n_steps, n_length = 20, 200
  n_features = 3
  n_outputs = 6

  print(testX.shape[0])
  testX = testX.reshape((testX.shape[0], 1, n_length, n_features))
  model = method_to_create_the_model(n_length, n_features, n_outputs)
  model.load_weights(fileweights)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  predictions = model.predict(testX)
  for pred in predictions:
    print(pred)
		

def compare_features(feature1, feature2):
	similarity = DWT(feature1, feature2)
	return similarity
		
def extract_features(testX):
	model = method_to_create_the_model(n_length, n_features)
	model.load_weights(fileweights)
  #controllare se -4 va bene
	extract = Model(model.inputs, model.layers[-4].output) 
	features_list = []
	for test in testX:
		features = extract.predict(test)
		save_features(features)
		features_list.append(features)
	return features_list

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(repeats=1):
	# load data
	trainX, trainy, testX, testy = load_WISDM_dataset('drive/MyDrive/Colab Notebooks/')
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
run_experiment()

#trainX, trainy, testX, testy = load_WISDM_dataset('drive/MyDrive/Colab Notebooks/')
#evaluate_network(testX)


(54901, 3, 200)
(43920, 200, 3)
>#1: 98.525
[98.5247254371643]
Accuracy: 98.525% (+/-0.000)


# Dati estratti con Early Stopping = loss

run_experiment() del 04/02

(54901, 3, 200)
(43920, 200, 3)
>#1: 98.525
[98.5247254371643]

Accuracy: 98.525% (+/-0.000)

In [ ]:
###_----main ---###
features_bianca = extract_features(bianca_walking)
features_giacomo = extract_features(giacomo_walking)

compare_features(features_bianca[0], features_bianca[1]) # similarità alta
compare_features(features_bianca[0], features_giacomo[1]) # similarità bassa

In [ ]:
1) Output

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 90.058
>#2: 91.144
>#3: 91.347
>#4: 90.227
>#5: 91.483
>#6: 90.058
>#7: 90.329
>#8: 89.345
>#9: 91.924
>#10: 90.804
[90.05768299102783, 91.14353656768799, 91.34713411331177, 90.22734761238098, 91.48286581039429, 90.05768299102783, 90.32914638519287, 89.34509754180908, 91.923987865448, 90.80420732498169]
Accuracy: 90.672% (+/-0.758)
